In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
import importlib
import time
import cv2
DATASET_PATH = '../dataset'
if DATASET_PATH not in sys.path:
    sys.path.append(DATASET_PATH)

不知道为啥算出来的c1Tc2矩阵不对

In [43]:
def get_delta_pose(c1Tw, c2Tw):
    # cTw = world to camera pose [4x4 matrix]
    # return = c2Tc1, which means c1 to c2 pose
    c1Rw = tf.slice(c1Tw, [0,0], [3,3])
    c2Rw = tf.slice(c2Tw, [0,0], [3,3])
    c1Pw = tf.slice(c1Tw, [0,3], [3,1])
    c2Pw = tf.slice(c2Tw, [0,3], [3,1])
    wPc1 = -tf.matmul(c1Rw, c1Pw, transpose_a=True) # wPc = -wRc cPw
    wPc2 = -tf.matmul(c2Rw, c2Pw, transpose_a=True) # wPc = -wRc cPw
    c2Rc1 = tf.matmul(c2Rw, c1Rw, transpose_b=True) # c2Rc1 = c2Rw wRc1
    c2Pc1 = tf.matmul(c2Rw, wPc1-wPc2) # c2Pc1 = c2Rw (wPc1-wPc2)
    # c2Tc1 (4x4) = 
    # | c2Rc1 c2Pc1 |
    # |   0     1   |
    c2Tc1 = tf.concat([c2Rc1, c2Pc1], axis=1)
    c2Tc1 = tf.concat([c2Tc1, tf.constant([[0,0,0,1]], dtype=tf.float32)], axis=0)
    c1Tc2 = tf.matrix_inverse(c2Tc1)
    return c2Tc1, c1Tc2

In [2]:
a = np.loadtxt('../dataset/pose/0.txt')

In [3]:
b = np.loadtxt('../dataset/pose/100.txt')

In [41]:
c1Rw = a[:3,:3]
c2Rw = b[:3,:3]
c1Pw = np.array([a[:3,3]]).T
c2Pw = np.array([b[:3,3]]).T

In [42]:
c1Rw

array([[-9.55421e-01,  1.19616e-01, -2.69932e-01],
       [ 2.95248e-01,  3.88339e-01, -8.72939e-01],
       [ 4.08000e-04, -9.13720e-01, -4.06343e-01]])

In [6]:
c2Rw.shape

(3, 3)

In [7]:
wPc1=-np.matmul(c1Rw.T,c1Pw)

In [8]:
wPc2=-np.matmul(c2Rw.T,c2Pw)

In [9]:
c2Rc1 = np.matmul(c2Rw, c1Rw.T) # c2Rc1 = c2Rw wRc1

In [10]:
c2Pc1 = np.matmul(c2Rw, wPc1-wPc2) # c2Pc1 = c2Rw (wPc1-wPc2)

In [11]:
c2Pc1

array([[-0.19752763],
       [ 0.41206409],
       [ 0.70960995]])

In [12]:
c2Tc1 = np.concatenate((c2Rc1, c2Pc1), axis=1)

In [13]:
c2Tc1

array([[ 0.99504734,  0.08814556,  0.0459535 , -0.19752763],
       [-0.09445814,  0.98245171,  0.16083608,  0.41206409],
       [-0.03096968, -0.16438074,  0.9859096 ,  0.70960995]])

In [14]:
c2Tc1 = np.concatenate([c2Tc1, np.array([[0,0,0,1]], dtype=np.float32)], axis=0)

In [15]:
c1Tc2 = np.linalg.inv(c2Tc1)

In [16]:
c1Tc2

array([[ 0.99504673, -0.0944574 , -0.0309701 ,  0.25744842],
       [ 0.08814612,  0.98244969, -0.16438016, -0.27077514],
       [ 0.0459533 ,  0.16083675,  0.98591182, -0.75681084],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [17]:
b

array([[-0.924645,  0.111265, -0.364214,  2.770857],
       [ 0.380379,  0.223266, -0.897477,  3.310602],
       [-0.018542, -0.968386, -0.248764,  1.486608],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [18]:
a

array([[-9.554210e-01,  1.196160e-01, -2.699320e-01,  2.655830e+00],
       [ 2.952480e-01,  3.883390e-01, -8.729390e-01,  2.981598e+00],
       [ 4.080000e-04, -9.137200e-01, -4.063430e-01,  1.368648e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

In [19]:
a.dot(c1Tc2)-b

array([[-0.02790412,  0.0530833 ,  0.10801184, -0.1891002 ],
       [-0.0924773 , -0.0100315 , -0.03614297,  0.30250428],
       [-0.0802657 ,  0.00530864, -0.00166956,  0.43708249],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [21]:
b.dot(c2Tc1)-a

array([[ 0.03612516, -0.03193709, -0.11374533,  0.08506837],
       [ 0.0899524 ,  0.01206572,  0.04149678, -0.29099007],
       [ 0.08031791,  0.00158514,  0.0044807 , -0.45393994],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [37]:
c1Rw = tf.slice(a, [0,0], [3,3])
c2Rw = tf.slice(b, [0,0], [3,3])
c1Pw = tf.slice(a, [0,3], [3,1])
c2Pw = tf.slice(b, [0,3], [3,1])

In [38]:
c1Rw

<tf.Tensor 'Slice_8:0' shape=(3, 3) dtype=float64>

In [39]:
c1Pw

<tf.Tensor 'Slice_10:0' shape=(3, 1) dtype=float64>

In [40]:
with tf.Session() as sess:
    print(sess.run(c1Rw))
    
        c1Rw = tf.slice(c1Tw, [0,0], [3,3])
    c2Rw = tf.slice(c2Tw, [0,0], [3,3])
    c1Pw = tf.slice(c1Tw, [0,3], [3,1])
    c2Pw = tf.slice(c2Tw, [0,3], [3,1])
    wPc1 = -tf.matmul(c1Rw, c1Pw, transpose_a=True) # wPc = -wRc cPw
    wPc2 = -tf.matmul(c2Rw, c2Pw, transpose_a=True) # wPc = -wRc cPw
    c2Rc1 = tf.matmul(c2Rw, c1Rw, transpose_b=True) # c2Rc1 = c2Rw wRc1
    c2Pc1 = tf.matmul(c2Rw, wPc1-wPc2) # c2Pc1 = c2Rw (wPc1-wPc2)
    # c2Tc1 (4x4) = 
    # | c2Rc1 c2Pc1 |
    # |   0     1   |
    c2Tc1 = tf.concat([c2Rc1, c2Pc1], axis=1)
    c2Tc1 = tf.concat([c2Tc1, tf.constant([[0,0,0,1]], dtype=tf.float32)], axis=0)
    c1Tc2 = tf.matrix_inverse(c2Tc1)

[[-9.55421e-01  1.19616e-01 -2.69932e-01]
 [ 2.95248e-01  3.88339e-01 -8.72939e-01]
 [ 4.08000e-04 -9.13720e-01 -4.06343e-01]]


In [59]:
c1Rw = tf.slice(a, [0,0], [3,3])
c2Rw = tf.slice(b, [0,0], [3,3])
c1Pw = tf.slice(a, [0,3], [3,1])
c2Pw = tf.slice(b, [0,3], [3,1])
wPc1 = -tf.matmul(c1Rw, c1Pw, transpose_a=True) # wPc = -wRc cPw
wPc2 = -tf.matmul(c2Rw, c2Pw, transpose_a=True) # wPc = -wRc cPw
c2Rc1 = tf.matmul(c2Rw, c1Rw, transpose_b=True) # c2Rc1 = c2Rw wRc1
c2Pc1 = tf.matmul(c2Rw, wPc1-wPc2) # c2Pc1 = c2Rw (wPc1-wPc2)

c2Tc1 = tf.concat([c2Rc1, c2Pc1], axis=1)
c2Tc1 = tf.concat([c2Tc1, tf.constant([[0,0,0,1]], dtype=tf.float64)], axis=0)

c1Tc2 = tf.matrix_inverse(c2Tc1)

with tf.Session() as sess:
    print(sess.run(c1Tc2))


[[ 0.99504673 -0.0944574  -0.0309701   0.25744842]
 [ 0.08814612  0.98244969 -0.16438016 -0.27077514]
 [ 0.0459533   0.16083675  0.98591182 -0.75681084]
 [ 0.          0.          0.          1.        ]]


In [57]:
    with tf.Session() as sess:
    data_numpy = data_tensor.eval()

AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
array([[ 0.99504673, -0.0944574 , -0.0309701 ,  0.25744842],
       [ 0.08814612,  0.98244969, -0.16438016, -0.27077514],
       [ 0.0459533 ,  0.16083675,  0.98591182, -0.75681084],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])